In [ ]:
data1 = [
    (1,"a1","BSAS"),
    (2,"a2","BSAS"),
    (3,"a3","BSAS"),
    (4,"a4","Noruega")
]

data2 = [
    ("2018-08-12", 1, 30),
    ("2018-08-14", 4, 30),
    ("2018-07-13", 1, 30),
    ("2018-08-14", 1, 30),
    ("2018-08-14", 1, 30),
    ("2017-07-12", 1, 30),
    ("2018-08-12", 1, 30),
    ("2018-08-12", 1, 30),
]

In [ ]:
#rdd1 - id_base , nombre , pcia , ciudad , lat , lon
rdd1 = sc.parallelize(data1);

#rdd2 - TIMESTAMP, ID_BASE, TEMPERATURA, HUMEDAD, 
#       PRESIÓN, DIRECCIÓN VIENTO, VELOCIDAD VIENTO
rdd2 = sc.parallelize(data2);

#ID y nombre result final

In [ ]:
rdd1 = rdd1.filter(lambda x: x[2]=="BSAS")
rdd2 = rdd2.filter(lambda x: x[0]>="2017-12-00")

In [ ]:
rdd1.map(lambda x: (x[0],x[1]))
#rdd1 -> id_base, nombre

rdd2.map(lambda x: ((x[1],x[0][:7]),(x[2],1) ) )
#rdd2 -> (año-mes, id_base) , (temp, 1)

In [ ]:
rdd2.reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))
#rdd2 -> (año-mes, id_base) , (suma de temp, cant de mediciones)

In [ ]:
def mes_anterior(x):
    mes=int(x[5:7])
    anio=int(x[:3])
    mes=mes-1
    if mes==0:
        anio=anio-1
        mes=12
    return (str(anio)+"-"+str(mes))
    

In [ ]:
rdd2.flatMap(lambda x: [ (x[0][1], (x[1][0]/x[1][1],mes_anterior(x[0][0]) ) ),
                        (x[0][1], (x[1][0]/x[1][1],x[0][0]) ) ] )
#rdd2 -> id_base , (promedio, anio mes)

In [ ]:
rdd2.groupByKey(lambda x,y: len(x)==2 & x[0][0]/y[0][0]>0.3)
# id_base, ((prom, mes),(prom,mes_anterior))

In [ ]:
rdd1.join(rdd2)
rdd1.map(lambda x: (x[0],x[1]))